<a href="https://colab.research.google.com/github/Riasci/KomputasiIntelegensiaTasks/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [19]:
import pandas as pd
import numpy as np

# Define some sample financial news headlines and their sentiment labels
data = {
    "headline": [
        "The company reported record profits this quarter.",
        "Investors are worried about the rising inflation rates.",
        "Stock prices fell sharply after the disappointing earnings report.",
        "New product launch is expected to boost sales significantly.",
        "The merger has raised concerns among regulators.",
        "Market analysts predict a downturn in the next quarter.",
        "Company X's stock has outperformed its competitors.",
        "There is uncertainty in the market due to geopolitical tensions.",
        "The acquisition is seen as a positive move for growth.",
        "Experts are optimistic about the company's future performance."
    ],
    "label": [
        "positive",  # Record profits
        "negative",  # Rising inflation
        "negative",  # Disappointing earnings
        "positive",  # Boost in sales
        "negative",  # Concerns about merger
        "negative",  # Market downturn
        "positive",  # Outperformed competitors
        "negative",  # Market uncertainty
        "positive",  # Positive acquisition
        "positive"   # Optimistic performance
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Save the synthetic dataset to a CSV file
df.to_csv("synthetic_financial_news.csv", index=False)

print("Synthetic dataset created:")
print(df)


Synthetic dataset created:
                                            headline     label
0  Experts are optimistic about the company's fut...  positive
1  Stock prices fell sharply after the disappoint...  negative
2  Investors are worried about the rising inflati...  negative
3  There is uncertainty in the market due to geop...  negative
4  Company X's stock has outperformed its competi...  positive
5  Market analysts predict a downturn in the next...  negative
6  The company reported record profits this quarter.  positive
7  The acquisition is seen as a positive move for...  positive
8   The merger has raised concerns among regulators.  negative
9  New product launch is expected to boost sales ...  positive


In [20]:
# Import library yang diperlukan
from transformers import pipeline
import numpy as np

# Inisialisasi pipeline model
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Ambil headline dari DataFrame
headlines = df["headline"].tolist()

# Lakukan prediksi
predictions = pipe(headlines)

# Ekstrak label yang diprediksi
predicted_labels = [pred['label'] for pred in predictions]

# Hitung akurasi
accuracy = np.mean(np.array(predicted_labels) == np.array(df["label"]))

print(f"Akurasi Model: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Akurasi Model: 1.00


In [21]:
pip install transformers torch


In [22]:
import torch
from torch import nn
from transformers import DistilBertModel

class AttentionTransformer(nn.Module):
    def __init__(self, model_name="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"):
        super(AttentionTransformer, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained(model_name)
        self.fc = nn.Linear(self.distilbert.config.dim, 2)  # Assuming binary classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        # Get the outputs from DistilBERT
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the last hidden state
        hidden_state = outputs.last_hidden_state

        # Apply attention mechanism
        # Here, we can use the mean of the hidden states as a simple form of attention
        attention_output = torch.mean(hidden_state, dim=1)  # Mean pooling

        # Pass through the fully connected layer
        logits = self.fc(attention_output)

        # Apply softmax to get probabilities
        probabilities = self.softmax(logits)

        return probabilities

# Example of using the AttentionTransformer
if __name__ == "__main__":
    # Create a sample input
    model = AttentionTransformer()

    # Example input tensors (batch size of 1, sequence length of 10)
    input_ids = torch.tensor([[101, 2023, 2003, 1037, 4895, 2330, 1998, 2031, 2064, 102]])  # Tokenized input
    attention_mask = torch.tensor([[1] * 10])  # Attention mask

    # Forward pass through the model
    probabilities = model(input_ids, attention_mask)

    print("Predicted probabilities:", probabilities)


You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer

Predicted probabilities: tensor([[0.4755, 0.5245]], grad_fn=<SoftmaxBackward0>)


In [24]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer  # Use RobertaTokenizer instead of DistilBertTokenizer

# Load the synthetic dataset
df = pd.read_csv("synthetic_financial_news.csv")

# Map labels to integers
label_map = {"positive": 1, "negative": 0}
df['label'] = df['label'].map(label_map)

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Define the FinancialNewsDataset class (make sure you have this class defined)
class FinancialNewsDataset(Dataset):
    def __init__(self, headlines, labels, tokenizer, max_len):
        self.headlines = headlines
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.headlines)

    def __getitem__(self, idx):
        headline = self.headlines[idx]
        label = self.labels[idx]

        # Tokenize the headline
        encoding = self.tokenizer.encode_plus(
            headline,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create dataset and dataloader
dataset = FinancialNewsDataset(df['headline'].tolist(), df['label'].tolist(), tokenizer, max_len=10)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Initialize the AttentionTransformer model
model = AttentionTransformer()
model.eval()  # Set the model to evaluation mode

# Function to evaluate accuracy
def evaluate_model(model, dataloader):
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            # Get model predictions
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)

            correct_predictions += torch.sum(preds == labels).item()
            total_predictions += labels.size(0)

    return correct_predictions / total_predictions

# Calculate and print model accuracy
accuracy = evaluate_model(model, dataloader)
print(f"Model Accuracy with Attention Transformer: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 't

Model Accuracy with Attention Transformer: 0.60


In [17]:
# Step 1: Install necessary libraries
!pip install transformers datasets

# Step 2: Import libraries and load the model
from transformers import pipeline

# Load pre-trained sentiment-analysis model
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
classifier = pipeline("sentiment-analysis", model=model_name)

# Step 3: Classify a sample text
text = "Operating profit totaled EUR 9.4 mn, down from EUR 11.7 mn in 2004."
result = classifier(text)

# Print the result
print(f"Text: {text}")
print(f"Sentiment: {result[0]['label']}, Confidence: {result[0]['score']}")


Text: Operating profit totaled EUR 9.4 mn, down from EUR 11.7 mn in 2004.
Sentiment: negative, Confidence: 0.9987685084342957


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Step 1: Install the necessary libraries
!pip install transformers torch

# Step 2: Import the required libraries
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification

# Step 3: Load the model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Step 4: Define a function for sentiment analysis
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return "Positive" if predicted_class == 1 else "Negative"

# Step 5: Test the function
sample_texts = [
    "The company's earnings have exceeded expectations.",
    "The recent financial report indicates a significant loss."
]

for text in sample_texts:
    sentiment = analyze_sentiment(text)
    print(f"Text: {text}\nSentiment: {sentiment}\n")


Text: The company's earnings have exceeded expectations.
Sentiment: Negative

Text: The recent financial report indicates a significant loss.
Sentiment: Negative



In [16]:
import random
import pandas as pd
from sklearn.metrics import accuracy_score

# Step 1: Create a synthetic dataset
def create_synthetic_dataset(num_samples=100):
    positive_samples = [
        "The company has reported record profits this quarter.",
        "Investors are optimistic about the upcoming product launch.",
        "Sales have surged due to increased demand.",
        "The stock price has risen steadily over the past month.",
        "The management team is confident about future growth."
    ]

    negative_samples = [
        "The company faces legal challenges that could affect its revenue.",
        "Recent layoffs have raised concerns among investors.",
        "The financial outlook appears grim for the next quarter.",
        "The company has seen a decline in market share.",
        "Rising costs are impacting profitability."
    ]

    # Generate synthetic dataset
    dataset = []
    for _ in range(num_samples):
        if random.choice([True, False]):
            text = random.choice(positive_samples)
            label = 1  # Positive
        else:
            text = random.choice(negative_samples)
            label = 0  # Negative
        dataset.append((text, label))

    return pd.DataFrame(dataset, columns=["text", "label"])

# Step 2: Create the dataset
synthetic_dataset = create_synthetic_dataset(num_samples=100)

# Step 3: Evaluate the model on the synthetic dataset
predictions = []
for index, row in synthetic_dataset.iterrows():
    sentiment = analyze_sentiment(row['text'])
    predictions.append(1 if sentiment == "Positive" else 0)

# Step 4: Calculate accuracy
accuracy = accuracy_score(synthetic_dataset['label'], predictions)
print(f"Model Accuracy on Synthetic Dataset: {accuracy:.2f}")


Model Accuracy on Synthetic Dataset: 0.51


In [15]:
# Step 1: Install the necessary libraries
!pip install transformers torch scikit-learn

# Step 2: Import required libraries
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 3: Load the model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Step 4: Create a synthetic dataset
data = [
    ("The company posted record profits this quarter.", 1),   # Positive
    ("Investors are worried about the company's debt.", 0),   # Negative
    ("Market trends indicate a bullish outlook for next year.", 1),  # Positive
    ("The recent layoffs have raised concerns among employees.", 0),  # Negative
    ("New product launch received great feedback from consumers.", 1),  # Positive
    ("The stock price has dropped significantly in the last month.", 0),  # Negative
    ("Analysts predict strong growth for the upcoming fiscal year.", 1),  # Positive
    ("The recent scandal has tarnished the company's reputation.", 0),  # Negative
]

texts, labels = zip(*data)

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.25, random_state=42)

# Step 6: Define a function for sentiment analysis
#def predict_sentiment(texts):
#    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
#    with torch.no_grad():
#        outputs = model(**inputs)
#    logits = outputs.logits
#    predicted_classes = torch.argmax(logits, dim=1).numpy()
#    return predicted_classes

# Step 7: Predict sentiments on the test set
#y_pred = predict_sentiment(X_test)
# Predict sentiments for the test set and store results
y_pred = []
for text in X_test:
    sentiment = analyze_sentiment(text)  # Pastikan ini mengembalikan 0 atau 1
    y_pred.append(sentiment)  # Simpan hasil prediksi sebagai label numerik
    print(f"Text: {text}\nSentiment: {sentiment}\n")

# Step 8: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Text: Investors are worried about the company's debt.
Sentiment: Negative

Text: The stock price has dropped significantly in the last month.
Sentiment: Negative

Accuracy: 0.00%


In [14]:
# Step 4: Create a synthetic dataset
data = [
    ("The company posted record profits this quarter.", 1),   # Positive
    ("Investors are worried about the company's debt.", 0),   # Negative
    ("Market trends indicate a bullish outlook for next year.", 1),  # Positive
    ("The recent layoffs have raised concerns among employees.", 0),  # Negative
    ("New product launch received great feedback from consumers.", 1),  # Positive
    ("The stock price has dropped significantly in the last month.", 0),  # Negative
    ("Analysts predict strong growth for the upcoming fiscal year.", 1),  # Positive
    ("The recent scandal has tarnished the company's reputation.", 0),  # Negative
]

texts, labels = zip(*data)

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.25, random_state=42)

# Predict sentiments for the test set and store results
y_pred = []
for text in X_test:
    sentiment = analyze_sentiment(text)  # Assume analyze_sentiment returns 1 or 0
    y_pred.append(sentiment)
    print(f"Text: {text}\nSentiment: {sentiment}\n")

# Step 8: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Text: Investors are worried about the company's debt.
Sentiment: Negative

Text: The stock price has dropped significantly in the last month.
Sentiment: Negative

Accuracy: 0.00%


In [10]:
# Step 2: Import required libraries
import matplotlib.pyplot as plt
import numpy as np

# Step 3: Load the model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Step 4: Create a synthetic dataset (or use your dataset)
data = [
    ("The company posted record profits this quarter.", 1),   # Positive
    ("Investors are worried about the company's debt.", 0),   # Negative
    ("Market trends indicate a bullish outlook for next year.", 1),  # Positive
    ("The recent layoffs have raised concerns among employees.", 0),  # Negative
]

texts, labels = zip(*data)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# Step 5: Define a function for sentiment analysis with attention extraction
def predict_sentiment_with_attention(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)  # Output attention weights
    logits = outputs.logits
    predicted_classes = torch.argmax(logits, dim=1).numpy()

    return predicted_classes, outputs.attentions  # Return predicted classes and attention weights

# Step 6: Predict sentiments and extract attention weights on the test set
y_pred, attentions = predict_sentiment_with_attention(X_test)
# Step 8: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Accuracy: 100.00%
